# Tutoriel : Utilisation avancée de Hugging Face Transformers

Ce tutoriel couvre les fonctionnalités clés de Hugging Face Transformers, alignées avec votre présentation : les pipelines, les classes dynamiques `Auto*`, le fine-tuning de modèles pré-entraînés, et l'intégration multimodale.

## 1. Importation des bibliothèques

In [1]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset


C:\Users\jrabault\Documents\00 - CNRS PRO\01 - Projet\00 - Banc de test\brouillon\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Utilisation des pipelines Hugging Face

Les pipelines offrent une interface simplifiée pour utiliser des modèles pré-entraînés. Essayons la classification de texte et la traduction.

In [2]:
# Pipeline de classification de texte
classificateur = pipeline('text-classification', model='distilbert-base-uncased-finetuned-sst-2-english')
resultat = classificateur('This movie was absolutely fantastic!')[0]
print('Résultat de la classification :', resultat)

# Pipeline de traduction (Anglais vers Français)
traducteur = pipeline('translation_en_to_fr', model='t5-base')
traduction = traducteur('Transformers are revolutionizing NLP.')[0]['translation_text']
print('Traduction :', traduction)

Résultat de la classification : {'label': 'POSITIVE', 'score': 0.999874472618103}
Traduction : Les transformateurs révolutionnent la LNP.


## 3. Chargement dynamique avec AutoModel et AutoTokenizer

Les classes `Auto*` permettent de charger facilement n'importe quel modèle ou tokenizer à partir du hub Hugging Face.

In [3]:
# Chargement dynamique du tokenizer et du modèle
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
model = AutoModelForSequenceClassification.from_pretrained('bert-base-cased', num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 4. Préparation des données et tokenisation

Utilisons le jeu de données IMDB pour la classification des sentiments, et appliquons la tokenisation avec le tokenizer BERT.

In [4]:
# Chargement et tokenisation des données IMDB
dataset = load_dataset('imdb')

def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 50000/50000 [00:56<00:00, 888.65 examples/s] 


## 5. Fine-tuning du modèle avec Trainer

Nous allons entraîner le modèle BERT sur les données IMDB en utilisant `Trainer` et `TrainingArguments`.

In [5]:
# Configuration des arguments d'entraînement
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
)

# Création du Trainer et entraînement
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'].select(range(2000)),
    eval_dataset=tokenized_datasets['test'].select(range(500)),
)
trainer.train()

C:\Users\jrabault\Documents\00 - CNRS PRO\01 - Projet\00 - Banc de test\brouillon\.venv\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


KeyboardInterrupt: 

## 6. Exemple multimodal : Classification d'images avec CLIP

Utilisons le modèle CLIP pour associer du texte à une image, démontrant l'intégration multimodale.

In [7]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image

# Chargement du modèle et du processeur CLIP
clip_model = CLIPModel.from_pretrained('openai/clip-vit-base-patch32')
clip_processor = CLIPProcessor.from_pretrained('openai/clip-vit-base-patch32')

# Chargement d'une image et du texte
image = Image.open('img.png')
texte = ['Un chien', 'Un chat', 'Un paysage']

# Préparation des entrées et inférence
inputs = clip_processor(text=texte, images=image, return_tensors='pt', padding=True)
outputs = clip_model(**inputs)
logits_per_image = outputs.logits_per_image
probas = logits_per_image.softmax(dim=1)
print('Probabilités :', probas)

Probabilités : tensor([[9.9877e-01, 2.8467e-04, 9.4090e-04]], grad_fn=<SoftmaxBackward0>)


## 7. Génération de légendes d'images en français

Dans cette section, nous allons générer une légende pour une image en anglais, puis la traduire en français. Nous utilisons le modèle `BLIP` pour la génération de légendes et un modèle de traduction de Helsinki-NLP.

In [8]:
from transformers import pipeline
from PIL import Image

# Chargement de l'image
image_path = 'img.png'
image = Image.open(image_path)

# Définition de la fonction pour générer la légende
def generer_legende_en_francais(chemin_image):
    pipeline_legende_image = pipeline('image-to-text', model='Salesforce/blip-image-captioning-large')
    pipeline_traduction = pipeline('translation', model='Helsinki-NLP/opus-mt-en-fr')

    resultats_legende = pipeline_legende_image(chemin_image)
    legende_anglaise = resultats_legende[0]['generated_text']
    legende_francaise = pipeline_traduction(legende_anglaise, max_length=512)
    texte_francais = legende_francaise[0]['translation_text']

    return texte_francais

# Génération de la légende en français
legende_francaise = generer_legende_en_francais(image)
print('Légende générée :', legende_francaise)

C:\Users\jrabault\Documents\00 - CNRS PRO\01 - Projet\00 - Banc de test\brouillon\.venv\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
C:\Users\jrabault\Documents\00 - CNRS PRO\01 - Projet\00 - Banc de test\brouillon\.venv\Lib\site-packages\transformers\generation\utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Légende générée : il y a un chien blanc qui est assis dans l'herbe
